In [1]:
%load_ext autoreload
%autoreload 2
import time
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from estimation_fct import *

# Load model
from model import ModelClass

model = ModelClass() 

par = model.par
sol = model.sol
sim = model.sim


c:\Users\Peter\Design_retirement\model.py:135: RuntimeWarning: invalid value encountered in divide
  np.cumsum(S[::-1])[::-1] / S,


In [ ]:
model.solve()
model.simulate()

In [7]:
# Prepare data
mean, weights, moments = prepare_data(par)

In [8]:
rows_to_drop = np.arange(40+70+55, 40+70+55+15)
cols_to_drop = np.arange(40+70+55, 40+70+55+15)

mean_drop = np.delete(mean, rows_to_drop, axis=0)

weights_drop = np.delete(weights, rows_to_drop, axis=0)
weights_drop = np.delete(weights_drop, cols_to_drop, axis=1)


In [13]:
mean_drop.shape

(205,)

In [4]:
# Prepare SMM
theta_names = ("beta", "sigma", "gamma", "mu", "zeta", "r_a", "r_s")
theta_init = np.array([9.33727108e-01, 1.11070508e+00, 4.60048114e+00, 6.5, 7.4, 7.00022874e-02, 15.99642707e-03])

# Original bounds
orig_bounds = [(0.0, 1.0),   # beta
               (0.1, 6.0),   # sigma
               (0.1, 10.0),  # gamma
               (0.0, 15.0),  # mu
               (0.0, 15.0),  # zeta
               (0.001, 0.1),   # r_a_in
               (0.001, 0.1 )
               ] 

theta_init_scaled = scale_params(theta_init, orig_bounds)

In [ ]:
# Do SMM
objective = lambda theta: obj_func(theta, theta_names, mean_drop, weights_drop, model, orig_bounds, do_print=True)

res = minimize(
    objective, 
    theta_init_scaled,
    method='Nelder-Mead',
    bounds=[(0,1)] * len(theta_init_scaled),
    tol=1e-6,
    options={"maxiter":1000}
)


beta=0.934 sigma=1.111 gamma=4.600 mu=6.500 zeta=7.400 r_a=0.070 r_s=0.016 
Error = 4.12, Time = 22.9182 seconds
beta=0.980 sigma=1.111 gamma=4.600 mu=6.500 zeta=7.400 r_a=0.070 r_s=0.016 
Error = 645.99, Time = 22.6246 seconds
beta=0.934 sigma=1.161 gamma=4.600 mu=6.500 zeta=7.400 r_a=0.070 r_s=0.016 
Error = 11.25, Time = 22.7616 seconds
beta=0.934 sigma=1.111 gamma=4.826 mu=6.500 zeta=7.400 r_a=0.070 r_s=0.016 
Error = 4.41, Time = 22.6436 seconds
beta=0.934 sigma=1.111 gamma=4.600 mu=6.825 zeta=7.400 r_a=0.070 r_s=0.016 
Error = 4.21, Time = 22.8112 seconds
beta=0.934 sigma=1.111 gamma=4.600 mu=6.500 zeta=7.770 r_a=0.070 r_s=0.016 
Error = 3.81, Time = 22.6615 seconds
beta=0.934 sigma=1.111 gamma=4.600 mu=6.500 zeta=7.400 r_a=0.073 r_s=0.016 
Error = 4.62, Time = 22.4472 seconds
beta=0.934 sigma=1.111 gamma=4.600 mu=6.500 zeta=7.400 r_a=0.070 r_s=0.017 
Error = 4.09, Time = 22.6865 seconds
beta=0.887 sigma=1.125 gamma=4.665 mu=6.593 zeta=7.506 r_a=0.071 r_s=0.016 
Error = 13.51, Ti

In [ ]:
# Load the model with calibrated values
theta_names = ("beta", "sigma", "gamma", "mu", "zeta", "r_a", "r_s")
# theta_final = np.array([0.991, 1.126, 7.350, 9.986, 10.0, 0.7, 1.1 ])
theta_final = unscale_params(res.x, orig_bounds)
# theta_final = np.array([0.961, 1.127, 2.645, 6.657, 0.009, 0.041])
# theta_final = np.array([9.33727108e-01, 1.11070508e+00, 4.60048114e+00, 6.5    7.4, 7.00022874e-02, 15.99642707e-03])

# for i, name in enumerate(theta_names):
#     setattr(model.par, name, theta_final[i])

model.solve()
model.simulate()

In [ ]:
theta_final

In [ ]:
# plot the graph with calibrated values 
a_dict = {
    'hours': [np.nanmean(np.where(model.sim.ex == 1, model.sim.h, np.nan),axis=0)[:40], moments['hours']],
    'extensive': [np.mean(model.sim.ex, axis=0)[:40], moments['extensive']],
    'illiquid': [np.clip(np.mean(model.sim.s, axis=0), 0, None), moments['savings']],
    'liquid': [np.mean(model.sim.a, axis=0), moments['assets']]
}

# Define colors
simulated_color = "navy"  # Dark blue
empirical_color = "darkred"  # Dark red
ci_color = "lightcoral"  # Light red for confidence bands

for key, (simulated, empirical) in a_dict.items():
    plt.figure(figsize=(10, 5))
    
    x_vals = np.arange(len(empirical)) + par.start_age
    
    plt.plot(x_vals, simulated, label=f"Simulated {key.capitalize()}", marker="o", color=simulated_color)
    plt.plot(x_vals, empirical, label=f"Empirical {key.capitalize()}", linestyle="--", marker="s", color=empirical_color)

    # # 99.9% confidence interval
    # ci = std_dev
    # plt.fill_between(x_vals, empirical - ci, empirical + ci, color=ci_color, alpha=0.4, label="Empirical Standard Deviation")

    plt.xlabel("Age")
    plt.ylabel(key.capitalize())
    plt.title(f"Comparison of Simulated and Empirical {key.capitalize()}")
    plt.legend()
    plt.grid(True)
    plt.show()


Old best variables:

In [ ]:
theta_old = ([0.982, 1.060, 3.877, 7.814, 0.02, 0.009])